In [ ]:
!git clone https://github.com/kstathou/vector_engine

fatal: destination path 'vector_engine' already exists and is not an empty directory.


In [ ]:
cd vector_engine

/content/vector_engine


In [ ]:
!pip install -r requirements.txt

Obtaining file:///content/vector_engine (from -r requirements.txt (line 9))
     |████████████████████████████████| 804.1 MB 2.9 kB/s 
     |████████████████████████████████| 1.1 MB 42.4 MB/s 
     |████████████████████████████████| 66 kB 4.9 MB/s 
     |████████████████████████████████| 10.5 MB 37.8 MB/s 
     |████████████████████████████████| 7.1 MB 14.4 MB/s 
     |████████████████████████████████| 14.5 MB 31 kB/s 
     |████████████████████████████████| 69 kB 7.1 MB/s 
     |████████████████████████████████| 7.1 MB 20.1 MB/s 
     |████████████████████████████████| 1.2 MB 34.8 MB/s 
     |████████████████████████████████| 895 kB 54.6 MB/s 
     |████████████████████████████████| 3.0 MB 34.7 MB/s 
     |████████████████████████████████| 4.3 MB 34.8 MB/s 
     |████████████████████████████████| 76 kB 4.6 MB/s 
     |████████████████████████████████| 131 kB 51.8 MB/s 
     |████████████████████████████████| 8.0 MB 39.7 MB/s 
     |████████████████████████████████| 111 kB 52.4 MB/s 
 

In [ ]:
!cat requirements.txt

torch==1.8.1
transformers==3.3.1
sentence-transformers==0.3.8
pandas==1.1.2
faiss-cpu==1.6.1
numpy==1.19.2
folium==0.2.1
streamlit==0.62.0
-e .

In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.1 MB/s 
     |████████████████████████████████| 2.9 MB 13.0 MB/s 
     |████████████████████████████████| 3.3 MB 38.1 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 636 kB 43.7 MB/s 
     |████████████████████████████████| 831.4 MB 2.3 kB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=a68bbeaad73d9dddb17370ae8877b878fb80903d0c3565c4122a686ebfe82f99
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: torch
    Found existing installation: torch 1.8.1
    Uninstalling torch-1.8.1:
      Successfully uninstalled torch-1.8.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc2
    Uninstalling tokenizers-0.8.1rc2:
      Successfully uninstalled tokenizers-0.8.1rc2
 

In [ ]:
!pip install s3fs

     |████████████████████████████████| 125 kB 4.2 MB/s 
     |████████████████████████████████| 52 kB 1.0 MB/s 
     |████████████████████████████████| 7.7 MB 12.1 MB/s 
     |████████████████████████████████| 1.3 MB 45.7 MB/s 
     |████████████████████████████████| 294 kB 51.9 MB/s 
     |████████████████████████████████| 142 kB 46.6 MB/s 
  Created wheel for aiobotocore: filename=aiobotocore-1.4.1-py3-none-any.whl size=49840 sha256=1a8db621fa9649d3b54c39528e475df0325433012ab84e8405c003925ae78575
  Stored in directory: /root/.cache/pip/wheels/25/3a/78/e9b2629d550d1598aa7f95648277711ca46aeb6ae846b90c69
Successfully built aiobotocore
  Attempting uninstall: botocore
    Found existing installation: botocore 1.21.53
    Uninstalling botocore-1.21.53:
      Successfully uninstalled botocore-1.21.53
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.18.53 re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Used to import data from S3.
import pandas as pd
import s3fs

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

# Used to do vector searches and display the results.
from vector_engine.utils import vector_search, id2details
from sklearn.metrics.pairwise import cosine_similarity
from spacy.lang.en import English # updated

In [ ]:
def find_esg_score(user_query):
    # Instantiate the sentence-level DistilBERT
    # model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
    model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
    # Check if GPU is available and use it
    if torch.cuda.is_available():
        model = model.to(torch.device("cuda"))
    print(model.device)

    ### User Query
    user_query = "Eco Friendly crackers are a thing of past. Now we are into no cracker zone."
    sentences = split_sentences(raw_text=user_query)
    query_embeddings = []
    for sentence in sentences:
        query_embeddings.append(model.encode(sentence))
    query_embedding = model.encode(user_query)
    k_val = 10

    from google.colab import drive
    drive.mount('/content/drive')

    with open('/content/drive/My Drive/ESG_Embeddings/new_esg_embeddings.npy', 'rb') as fp:
        esg_embeddings = np.load(fp)

    # Step 1: Instantiate the index
    esg_index = faiss.IndexFlatL2(esg_embeddings.shape[1])

    # Step 2: Pass the index to IndexIDMap
    esg_index = faiss.IndexIDMap(esg_index)

    # Step 3: Add vectors and their IDs
    values = [i for i in range(esg_embeddings.shape[0])]
    esg_index.add_with_ids(esg_embeddings, np.array(values))

    print(f"Number of vectors in the Faiss index: {esg_index.ntotal}")

    # For convenience, I've wrapped all steps in the vector_search function.
    # It takes four arguments: 
    # A query, the sentence-level transformer, the Faiss index and the number of requested results
    D_esg_embeddings, I_esg_embeddings = vector_search([user_query], model, esg_index, num_results=k_val)
    print(f'L2 distance: {D_esg_embeddings.flatten().tolist()}\n\nMAG paper IDs: {I_esg_embeddings.flatten().tolist()}')

    cosine_similarity_val = 0
    I_esg_embeddings = I_esg_embeddings.squeeze(0)
    query_embedding = query_embedding.reshape(1, -1)
    for index in I_esg_embeddings:
        esg_embedding = esg_embeddings[index]
        esg_embedding = esg_embedding.reshape(1, -1)
        query_embedding = query_embedding
        cosine_similarity_local = cosine_similarity(
            query_embedding,
            esg_embedding
        )
        print(cosine_similarity_local)
        cosine_similarity_val += cosine_similarity_local
    mean_knn_value_esg = (float)(cosine_similarity_val / k_val)

    with open('/content/drive/My Drive/ESG_Embeddings/non_esg_embeddings.npy', 'rb') as fp:
        not_esg_embeddings = np.load(fp)

    # Step 1: Instantiate the index
    not_esg_index = faiss.IndexFlatL2(not_esg_embeddings.shape[1])

    # Step 2: Pass the index to IndexIDMap
    not_esg_index = faiss.IndexIDMap(not_esg_index)

    # Step 3: Add vectors and their IDs
    not_esg_values = [i for i in range(not_esg_embeddings.shape[0])]
    not_esg_index.add_with_ids(not_esg_embeddings, np.array(not_esg_values))

    print(f"Number of vectors in the Faiss index: {not_esg_index.ntotal}")

    D_not_esg_embeddings, I_not_esg_embeddings = vector_search([user_query], model, not_esg_index, num_results=k_val)
    print(f'L2 distance: {D_not_esg_embeddings.flatten().tolist()}\n\nMAG paper IDs: {I_not_esg_embeddings.flatten().tolist()}')

    cosine_similarity_val = 0
    I_not_esg_embeddings = I_not_esg_embeddings.squeeze(0)
    for index in I_not_esg_embeddings:
        not_esg_embedding = not_esg_embeddings[index]
        not_esg_embedding = not_esg_embedding.reshape(1, -1)
        cosine_similarity_local = cosine_similarity(
            query_embedding,
            not_esg_embedding
        )
        print(cosine_similarity_local)
        cosine_similarity_val += cosine_similarity_local
    mean_knn_value_not_esg = (float)(cosine_similarity_val / k_val)
    mean_knn_value_not_esg

    if mean_knn_value_esg > mean_knn_value_not_esg:
        st.write(f"The given sentence has positive ESG score")
    else:
        st.write(f"The given sentence is not ESG oriented")

In [ ]:
# Instantiate the sentence-level DistilBERT
# model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

cpu


In [ ]:
def split_sentences(raw_text=None):
    if raw_text:
        raw_text = 'Hello, world. Here are two sentences.'
        nlp = English()
        nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
        doc = nlp(raw_text)
        sentences = [sent.string.strip() for sent in doc.sents]
        return sentences
    return None

In [ ]:
### User Query
user_query = "Eco Friendly crackers are a thing of past. Now we are into no cracker zone."
sentences = split_sentences(raw_text=user_query)
query_embeddings = []
for sentence in sentences:
    query_embeddings.append(model.encode(sentence))
query_embedding = model.encode(user_query)
k_val = 10

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [ ]:
len(query_embeddings)

2

In [ ]:
query_embeddings[0].shape

(768,)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/ESG_Embeddings/new_esg_embeddings.npy', 'rb') as fp:
    esg_embeddings = np.load(fp)

In [ ]:
esg_embeddings.shape

(223866, 768)

In [ ]:
# Step 1: Instantiate the index
esg_index = faiss.IndexFlatL2(esg_embeddings.shape[1])

# Step 2: Pass the index to IndexIDMap
esg_index = faiss.IndexIDMap(esg_index)

# Step 3: Add vectors and their IDs
values = [i for i in range(esg_embeddings.shape[0])]
esg_index.add_with_ids(esg_embeddings, np.array(values))

print(f"Number of vectors in the Faiss index: {esg_index.ntotal}")

Number of vectors in the Faiss index: 223866


In [ ]:
# # Retrieve the 10 nearest neighbours
# D, I = esg_index.search(np.array([esg_embeddings[5415]]), k=10)
# print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

In [ ]:
# For convenience, I've wrapped all steps in the vector_search function.
# It takes four arguments: 
# A query, the sentence-level transformer, the Faiss index and the number of requested results
D_esg_embeddings, I_esg_embeddings = vector_search([user_query], model, esg_index, num_results=k_val)
print(f'L2 distance: {D_esg_embeddings.flatten().tolist()}\n\nMAG paper IDs: {I_esg_embeddings.flatten().tolist()}')

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


L2 distance: [153.6286163330078, 153.75076293945312, 156.06871032714844, 157.23638916015625, 158.6468505859375, 159.37130737304688, 162.56158447265625, 162.56158447265625, 162.7227783203125, 163.05404663085938]

MAG paper IDs: [22843, 46513, 45670, 51574, 88546, 25257, 201736, 201735, 24680, 143527]


In [ ]:
cosine_similarity_val = 0
I_esg_embeddings = I_esg_embeddings.squeeze(0)
query_embedding = query_embedding.reshape(1, -1)
for index in I_esg_embeddings:
    esg_embedding = esg_embeddings[index]
    esg_embedding = esg_embedding.reshape(1, -1)
    query_embedding = query_embedding
    cosine_similarity_local = cosine_similarity(
        query_embedding,
        esg_embedding
    )
    print(cosine_similarity_local)
    cosine_similarity_val += cosine_similarity_local
mean_knn_value_esg = (float)(cosine_similarity_val / k_val)
mean_knn_value_esg

[[0.6884649]]
[[0.684738]]
[[0.6808002]]
[[0.6840043]]
[[0.6763419]]
[[0.6753343]]
[[0.6775511]]
[[0.6775511]]
[[0.66554344]]
[[0.6706737]]


0.6781003475189209

In [ ]:
with open('/content/drive/My Drive/ESG_Embeddings/non_esg_embeddings.npy', 'rb') as fp:
    not_esg_embeddings = np.load(fp)

In [ ]:
# Step 1: Instantiate the index
not_esg_index = faiss.IndexFlatL2(not_esg_embeddings.shape[1])

# Step 2: Pass the index to IndexIDMap
not_esg_index = faiss.IndexIDMap(not_esg_index)

# Step 3: Add vectors and their IDs
not_esg_values = [i for i in range(not_esg_embeddings.shape[0])]
not_esg_index.add_with_ids(not_esg_embeddings, np.array(not_esg_values))

print(f"Number of vectors in the Faiss index: {not_esg_index.ntotal}")

Number of vectors in the Faiss index: 94465


In [ ]:
# For convenience, I've wrapped all steps in the vector_search function.
# It takes four arguments: 
# A query, the sentence-level transformer, the Faiss index and the number of requested results
D_not_esg_embeddings, I_not_esg_embeddings = vector_search([user_query], model, not_esg_index, num_results=k_val)
print(f'L2 distance: {D_not_esg_embeddings.flatten().tolist()}\n\nMAG paper IDs: {I_not_esg_embeddings.flatten().tolist()}')

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


L2 distance: [147.01564025878906, 164.49513244628906, 168.19338989257812, 169.11575317382812, 169.21640014648438, 170.76544189453125, 170.79855346679688, 172.89617919921875, 174.07803344726562, 176.05526733398438]

MAG paper IDs: [77160, 81428, 65963, 91717, 55542, 55543, 35752, 60588, 70679, 8431]


In [ ]:
cosine_similarity_val = 0
I_not_esg_embeddings = I_not_esg_embeddings.squeeze(0)
for index in I_not_esg_embeddings:
    not_esg_embedding = not_esg_embeddings[index]
    not_esg_embedding = not_esg_embedding.reshape(1, -1)
    cosine_similarity_local = cosine_similarity(
        query_embedding,
        not_esg_embedding
    )
    print(cosine_similarity_local)
    cosine_similarity_val += cosine_similarity_local
mean_knn_value_not_esg = (float)(cosine_similarity_val / k_val)
mean_knn_value_not_esg

[[0.6960606]]
[[0.67056775]]
[[0.64748377]]
[[0.6538496]]
[[0.6477086]]
[[0.6449932]]
[[0.6582215]]
[[0.65454817]]
[[0.63876307]]
[[0.6418191]]


0.6554015278816223

In [ ]:
if mean_knn_value_esg > mean_knn_value_not_esg:
    print("The given sentence has positive ESG score")
else:
    print("The given sentence is not ESG oriented")

The given sentence has positive ESG score


In [ ]:
!pip install streamlit
!pip install pyngrok
from pyngrok import ngrok

  Using cached botocore-1.21.53-py3-none-any.whl (8.0 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.20.106
    Uninstalling botocore-1.20.106:
      Successfully uninstalled botocore-1.20.106
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 1.4.1 requires botocore<1.20.107,>=1.20.106, but you have botocore 1.21.53 which is incompatible.
     |████████████████████████████████| 745 kB 4.2 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=e86abc59658d63f7fc82a712a7db8e64ce6e6008109df87d4058b55b4f165136
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
# Used to import data from S3.
import pandas as pd
import s3fs

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

# Used to do vector searches and display the results.
from vector_engine.utils import vector_search, id2details
from sklearn.metrics.pairwise import cosine_similarity
from spacy.lang.en import English # updated

def split_sentences(raw_text=None):
    if raw_text:
        raw_text = 'Hello, world. Here are two sentences.'
        nlp = English()
        nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
        doc = nlp(raw_text)
        sentences = [sent.string.strip() for sent in doc.sents]
        return sentences
    return None

def find_esg_score(user_query):
    # Instantiate the sentence-level DistilBERT
    # model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
    model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
    # Check if GPU is available and use it
    if torch.cuda.is_available():
        model = model.to(torch.device("cuda"))
    print(model.device)

    ### User Query
    # user_query = "Eco Friendly crackers are a thing of past. Now we are into no cracker zone."
    sentences = split_sentences(raw_text=user_query)
    query_embeddings = []
    for sentence in sentences:
        query_embeddings.append(model.encode(sentence))
    query_embedding = model.encode(user_query)
    k_val = 3

    from google.colab import drive
    drive.mount('/content/drive')

    with open('/content/drive/My Drive/ESG_Embeddings/new_esg_embeddings.npy', 'rb') as fp:
        esg_embeddings = np.load(fp)

    # with open('/content/drive/My Drive/ESG_Embeddings/esg_word_embeddings.npy', 'rb') as fp:
    #     esg_embeddings = np.load(fp)

    esg_embeddings = esg_embeddings[:94465]
    # Step 1: Instantiate the index
    esg_index = faiss.IndexFlatL2(esg_embeddings.shape[1])

    # Step 2: Pass the index to IndexIDMap
    esg_index = faiss.IndexIDMap(esg_index)

    # Step 3: Add vectors and their IDs
    values = [i for i in range(esg_embeddings.shape[0])]
    esg_index.add_with_ids(esg_embeddings, np.array(values))

    print(f"Number of vectors in the Faiss index: {esg_index.ntotal}")

    # For convenience, I've wrapped all steps in the vector_search function.
    # It takes four arguments: 
    # A query, the sentence-level transformer, the Faiss index and the number of requested results
    D_esg_embeddings, I_esg_embeddings = vector_search([user_query], model, esg_index, num_results=k_val)
    print(f'L2 distance: {D_esg_embeddings.flatten().tolist()}\n\nMAG paper IDs: {I_esg_embeddings.flatten().tolist()}')

    cosine_similarity_val = 0
    I_esg_embeddings = I_esg_embeddings.squeeze(0)
    query_embedding = query_embedding.reshape(1, -1)
    for index in I_esg_embeddings:
        esg_embedding = esg_embeddings[index]
        esg_embedding = esg_embedding.reshape(1, -1)
        query_embedding = query_embedding
        cosine_similarity_local = cosine_similarity(
            query_embedding,
            esg_embedding
        )
        print(cosine_similarity_local)
        cosine_similarity_val += cosine_similarity_local
    mean_knn_value_esg = (float)(cosine_similarity_val / k_val)

    with open('/content/drive/My Drive/ESG_Embeddings/non_esg_embeddings.npy', 'rb') as fp:
        not_esg_embeddings = np.load(fp)

    # with open('/content/drive/My Drive/ESG_Embeddings/non_esg_word_embeddings.npy', 'rb') as fp:
    #     not_esg_embeddings = np.load(fp)

    # Step 1: Instantiate the index
    not_esg_index = faiss.IndexFlatL2(not_esg_embeddings.shape[1])

    # Step 2: Pass the index to IndexIDMap
    not_esg_index = faiss.IndexIDMap(not_esg_index)

    # Step 3: Add vectors and their IDs
    not_esg_values = [i for i in range(not_esg_embeddings.shape[0])]
    not_esg_index.add_with_ids(not_esg_embeddings, np.array(not_esg_values))

    print(f"Number of vectors in the Faiss index: {not_esg_index.ntotal}")

    D_not_esg_embeddings, I_not_esg_embeddings = vector_search([user_query], model, not_esg_index, num_results=k_val)
    print(f'L2 distance: {D_not_esg_embeddings.flatten().tolist()}\n\nMAG paper IDs: {I_not_esg_embeddings.flatten().tolist()}')

    cosine_similarity_val = 0
    I_not_esg_embeddings = I_not_esg_embeddings.squeeze(0)
    for index in I_not_esg_embeddings:
        not_esg_embedding = not_esg_embeddings[index]
        not_esg_embedding = not_esg_embedding.reshape(1, -1)
        cosine_similarity_local = cosine_similarity(
            query_embedding,
            not_esg_embedding
        )
        print(cosine_similarity_local)
        cosine_similarity_val += cosine_similarity_local
    mean_knn_value_not_esg = (float)(cosine_similarity_val / k_val)
    mean_knn_value_not_esg

    # if mean_knn_value_esg - mean_knn_value_not_esg > 0.015:
    if mean_knn_value_esg > mean_knn_value_not_esg:
        st.write(f"The given sentence has positive ESG score")
    else:
        st.write(f"The given sentence is not ESG oriented")
    # st.write(f"{mean_knn_value_esg}")
    # st.write(f"{mean_knn_value_not_esg}")
    # st.write(mean_knn_value_esg - mean_knn_value_not_esg)
st.title('ESG Score Computation')
st.text('Insert Text')

user_input = st.text_area("label goes here", None)

if st.button('Search'):
    st.write(f"{user_input}")
    find_esg_score(user_input)

Overwriting app.py


In [ ]:
!ps -ef | grep ngrok
!nohup streamlit run --server.port 80 app.py &

root         511      76  0 14:45 ?        00:00:00 /bin/bash -c ps -ef | grep ngrok
root         513     511  0 14:45 ?        00:00:00 grep ngrok
nohup: appending output to 'nohup.out'


In [ ]:
url = ngrok.connect(port='8501')

In [ ]:
!ps -ef | grep ngrok

root         520      75  0 11:16 ?        00:00:04 /usr/local/lib/python3.7/dist-packages/pyngrok/bin/ngrok start --none --log=stdout
root         790      75  0 11:37 ?        00:00:00 /bin/bash -c ps -ef | grep ngrok
root         792     790  0 11:37 ?        00:00:00 grep ngrok


In [ ]:
!kill 520

In [ ]:
url

<NgrokTunnel: "http://bbdf-35-234-45-176.ngrok.io" -> "http://localhost:80">

In [ ]:
len(not_esg_embeddings)

94465